# Correlated multi-omics features in SARS-Cov-2 infected samples

Examine the correlations returned by the integrative analysis.

Our hypothesis here is to test if there are features in multiomics data which are absent from singleomics data.

Steps:
- Flatten correlations for a more human readable format
- Preserve intraomics correlations? or split out into separate dataframes
- Also look at top loadings returned by splsda

In [1]:
import numpy as np
import pandas as pd

```
# feature names
# running these commands unites the features in each component
dir="../../splsdacentroids_diablomahalanobis_i24_comp4_splsdakeepx10_100_diablokeepx5_30_corrected/"
export dir

cd ../results/results/singleblock/proteome
(head -n1 ${dir}proteome_1_sPLSDA_max.txt; cat ${dir}proteome_*sPLSDA*max.txt | grep -v "importance") \
    > proteome_sPLSDA_keepx_max.txt

cd ../translatome
(head -n1 ${dir}translatome_1_sPLSDA_max.txt; cat ${dir}translatome_*sPLSDA*max.txt | grep -v "importance") \
    > translatome_sPLSDA_keepx_max.txt

cd ../../multiblock/
(head -n1 ${dir}proteome_1_DIABLO_var_keepx_max.txt; cat ${dir}proteome_*DIABLO*keepx_max*txt | grep -v "importance") \
    > proteome_DIABLO_keepx_max.txt
    
(head -n1 ${dir}translatome_1_DIABLO_var_keepx_max.txt; cat ${dir}translatome_*DIABLO*keepx_max*txt | grep -v "importance") \
    > translatome_DIABLO_keepx_max.txt
    
# feature ids
# running these commands unites the features in each component
dir="../../splsdacentroids_diablomahalanobis_i24_comp4_splsdakeepx10_100_diablokeepx5_30_corrected_nomap/"
export dir

cd ../results/results/singleblock/proteome
(head -n1 ${dir}proteome_1_sPLSDA_max.txt; cat ${dir}proteome_*sPLSDA*max.txt | grep -v "importance") \
    > proteome_sPLSDA_keepx_max.txt

cd ../translatome
(head -n1 ${dir}translatome_1_sPLSDA_max.txt; cat ${dir}translatome_*sPLSDA*max.txt | grep -v "importance") \
    > translatome_sPLSDA_keepx_max.txt

cd ../../multiblock/
(head -n1 ${dir}proteome_1_DIABLO_var_keepx_max.txt; cat ${dir}proteome_*DIABLO*keepx_max*txt | grep -v "importance") \
    > proteome_DIABLO_keepx_max.txt
    
(head -n1 ${dir}translatome_1_DIABLO_var_keepx_max.txt; cat ${dir}translatome_*DIABLO*keepx_max*txt | grep -v "importance") \
    > translatome_DIABLO_keepx_max.txt
```

In [2]:
# prot_plsda_infile = "../results_ids/results/singleblock/proteome/proteome_1_PLSDA_max.txt"
prot_splsda_infile = "../results/results_ids/singleblock/proteome/proteome_sPLSDA_keepx_max.txt"
# tran_plsda_infile = "../results_ids/results/singleblock/translatome/translatome_1_PLSDA_max.txt"
tran_splsda_infile = "../results/results_ids/singleblock/translatome/translatome_sPLSDA_keepx_max.txt"
diablo_prot_infile = "../results/results_ids/multiblock/proteome_1_DIABLO_var_keepx_max.txt"
diablo_tran_infile = "../results/results_ids/multiblock/translatome_1_DIABLO_var_keepx_max.txt"
diablo_corr_infile = "../results/results_ids/multiblock/DIABLO_var_keepx_correlations.txt"
panther_annotations_infile = "../data/panther.txt"
mapping_annotations_infile = "../data/mapping.csv"
covid_annotations_infile = "../data/uniprot_SARS-COV2_annotations.tab"
prot_map_infile = "../data/proteome_mapfile.txt"
tran_map_infile = "../data/translatome_mapfile.txt"

# prot_plsda = pd.read_csv(prot_plsda_infile, sep="\t")
prot_splsda = pd.read_csv(prot_splsda_infile, sep="\t")
# tran_plsda = pd.read_csv(tran_plsda_infile, sep="\t")
tran_splsda = pd.read_csv(tran_splsda_infile, sep="\t")
diablo_prot = pd.read_csv(diablo_prot_infile, sep="\t")
diablo_tran = pd.read_csv(diablo_tran_infile, sep="\t")
diablo_corr = pd.read_csv(diablo_corr_infile, sep="\t")
panther_annotations = pd.read_csv(panther_annotations_infile, sep="\t", header=None)
covid_annotations = pd.read_csv(covid_annotations_infile, sep="\t")
mapping_annotations = pd.read_csv(mapping_annotations_infile, sep=",")
prot_map = pd.read_csv(prot_map_infile, sep="\t", usecols=["key", "val"])
tran_map = pd.read_csv(tran_map_infile, sep="\t", usecols=["key", "val"])

In [3]:
# flatten the correlations for a more human readable format
flat_corr = diablo_corr.stack().reset_index()

# order of cols doesnt matter here, matrix is symmetrical
prot = flat_corr.level_0.str.contains('_proteome')   
tran = flat_corr.level_1.str.contains('_translatome')
multiblock = flat_corr[prot & tran]

# below block proves the above point:
_prot = flat_corr.level_1.str.contains('_proteome')   
_tran = flat_corr.level_0.str.contains('_translatome')
_multiblock = flat_corr[_prot & _tran]

ver1 = multiblock.reset_index().drop("index", axis=1).sort_values(by=["level_0", "level_1"], ignore_index=True)
ver2 = _multiblock.reset_index().drop("index", axis=1)[["level_1", "level_0", 0]]
ver2.columns = ["level_0", "level_1", 0]
ver2 = ver2.sort_values(by=["level_0", "level_1"], ignore_index=True)

assert ver1[["level_0", "level_1"]].equals(ver2[["level_0", "level_1"]]) is True, \
    "Order of correlated features must be identical"
assert set(np.isclose(ver1[0], ver2[0])) == {True}, \
    "Order of correlated values are identical"

# we want correlations across modalities only for this step
prot_tran = flat_corr[(flat_corr.level_0.str.contains("_proteome")) & (flat_corr.level_1.str.contains("_translatome"))]
tran_prot = flat_corr[(flat_corr.level_1.str.contains("_proteome")) & (flat_corr.level_0.str.contains("_translatome"))]
prot_tran.columns = ["proteome", "translatome", "correlation"]
prot_tran = prot_tran.sort_values(by=["proteome", "translatome"], ignore_index=True)

# clean up file names
prot_tran = prot_tran.replace(regex={r'_prot_proteome$': '', r'_tran_translatome': ''})
prot_tran = prot_tran.replace(regex={r'_proteome$': '', r'_translatome': ''})

# use panther annotations
panther_annot = panther_annotations[[1,2,4]].replace(regex={r',.*$': ''})
panther_annot.columns = [1,2,3]

# use covid annotations
covid_annot = covid_annotations.copy()
covid_annot.replace(regex={r'_HUMAN$': ''}, inplace=True)
covid_annot.drop(["Status", "Gene names", "Entry name", "Length"], axis=1, inplace=True)
covid_annot.columns = [1, 2, 3]
covid_annot

# merge annotations
annot = pd.concat([panther_annot, covid_annot], ignore_index=True)
annot

prot_tran_annot = pd.merge(annot, prot_tran, how="right", left_on=1, right_on="proteome")
prot_tran_annot.columns = [
    "tmp2", "prot_annotation", "prot_category", "proteome", 
    "translatome", "correlation"
]
prot_tran_annot = pd.merge(annot, prot_tran_annot, how="right", left_on=1, right_on="translatome")
prot_tran_annot.columns = [
    "tmp1", "tran_annotation", "tran_category", "tmp2", 
    "prot_annotation", "prot_category", "proteome", 
    "translatome", "correlation"
]
prot_tran_annot.drop(["tmp1", "tmp2"], axis=1, inplace=True)
prot_tran_annot
cols = [
    "translatome", "tran_annotation", "tran_category", 
    "proteome", "prot_annotation", "prot_category", "correlation"
]
prot_tran_annot = prot_tran_annot[cols]
prot_tran_annot.sort_values(by="correlation", ascending=False, ignore_index=True, inplace=True)
prot_tran_annot.to_csv("multiblock_correlations_annotated.tsv", sep="\t")
prot_tran_annot

id_to_feature = mapping_annotations.iloc[:,0:2]
id_to_feature.columns = ["name", "id"]
id_to_feature
prot_tran_annot = pd.merge(id_to_feature, prot_tran_annot, how="right", left_on="id", right_on="proteome")
prot_tran_annot.columns = [
    "prot_name", "tmp1", "translatome", "tran_annotation", "tran_category", 
    "proteome", "prot_annotation", "prot_category", "correlation"
]
prot_tran_annot = pd.merge(id_to_feature, prot_tran_annot, how="right", left_on="id", right_on="translatome")
prot_tran_annot.columns = [
    "tran_name", "tmp2", "prot_name", "tmp1", "translatome", "tran_annotation", 
    "tran_category", "proteome", "prot_annotation", "prot_category", "correlation"
]
cols = ["translatome", "tran_name", "tran_annotation", "tran_category", "proteome", "prot_name", "prot_annotation", "prot_category", "correlation"]
prot_tran_annot = prot_tran_annot[cols]
prot_tran_annot.sort_values(by="correlation", ascending=False, ignore_index=True, inplace=True)
prot_tran_annot.to_csv("multiblock_correlations_annotated.tsv", sep="\t")
prot_tran_annot

,translatome,tran_name,tran_annotation,tran_category,proteome,prot_name,prot_annotation,prot_category,correlation
0,Q7Z3J2,C16orf62,VPS35 endosomal protein sorting factor-like;VP...,NaN,Q7Z3J2,C16orf62,VPS35 endosomal protein sorting factor-like;VP...,NaN,0.977400
1,Q00839,HNRNPU,Heterogeneous nuclear ribonucleoprotein U;HNRN...,NaN,P0DTC6,NaN,Non-structural protein 6 (ns6) (Accessory prot...,Severe acute respiratory syndrome coronavirus ...,0.973599
2,Q00839,HNRNPU,Heterogeneous nuclear ribonucleoprotein U;HNRN...,NaN,P0DTC1,NaN,Replicase polyprotein 1a (pp1a) (ORF1a polypro...,Severe acute respiratory syndrome coronavirus ...,0.971105
3,Q7Z3J2,C16orf62,VPS35 endosomal protein sorting factor-like;VP...,NaN,Q9NPE3,NOP10,H/ACA ribonucleoprotein complex subunit 3;NOP1...,RNA binding protein(PC00031),0.969983
4,Q07021,C1QBP,Complement component 1 Q subcomponent-binding ...,NaN,P0DTC6,NaN,Non-structural protein 6 (ns6) (Accessory prot...,Severe acute respiratory syndrome coronavirus ...,0.969632
...,...,...,...,...,...,...,...,...,...
6365,Q9Y281,NaN,NaN,NaN,Q15465,SHH,Sonic hedgehog protein;SHH;ortholog,NaN,-0.962754
6366,P47756,CAPZB,F-actin-capping protein subunit beta;CAPZB;ort...,non-motor actin binding protein(PC00165),Q99519,NEU1,Sialidase-1;NEU1;ortholog,hydrolase(PC00121),-0.963688
6367,Q9NX63,CHCHD3,MICOS complex subunit MIC19;CHCHD3;ortholog,NaN,Q99519,NEU1,Sialidase-1;NEU1;ortholog,hydrolase(PC00121),-0.972778
6368,P47756,CAPZB,F-actin-capping protein subunit beta;CAPZB;ort...,non-motor actin binding protein(PC00165),Q15465,SHH,Sonic hedgehog protein;SHH;ortholog,NaN,-0.977356


# This block works if the data contains feature names (not unique ids!)

# flatten the correlations for a more human readable format
flat_corr = diablo_corr.stack().reset_index()

# order of cols doesnt matter here, matrix is symmetrical
prot = flat_corr.level_0.str.contains('_proteome')   
tran = flat_corr.level_1.str.contains('_translatome')
multiblock = flat_corr[prot & tran]

# below block proves the above point:
_prot = flat_corr.level_1.str.contains('_proteome')   
_tran = flat_corr.level_0.str.contains('_translatome')
_multiblock = flat_corr[_prot & _tran]

ver1 = multiblock.reset_index().drop("index", axis=1).sort_values(by=["level_0", "level_1"], ignore_index=True)
ver2 = _multiblock.reset_index().drop("index", axis=1)[["level_1", "level_0", 0]]
ver2.columns = ["level_0", "level_1", 0]
ver2 = ver2.sort_values(by=["level_0", "level_1"], ignore_index=True)

assert ver1[["level_0", "level_1"]].equals(ver2[["level_0", "level_1"]]) is True, \
    "Order of correlated features must be identical"
assert set(np.isclose(ver1[0], ver2[0])) == {True}, \
    "Order of correlated values are identical"

# we want correlations across modalities only for this step
prot_tran = flat_corr[(flat_corr.level_0.str.contains("_proteome")) & (flat_corr.level_1.str.contains("_translatome"))]
tran_prot = flat_corr[(flat_corr.level_1.str.contains("_proteome")) & (flat_corr.level_0.str.contains("_translatome"))]
prot_tran.columns = ["proteome", "translatome", "correlation"]
prot_tran = prot_tran.sort_values(by=["proteome", "translatome"], ignore_index=True)

# clean up file names
prot_tran = prot_tran.replace(regex={r'_prot_proteome$': '', r'_tran_translatome': ''})
prot_tran = prot_tran.replace(regex={r'_proteome$': '', r'_translatome': ''})
prot_tran = prot_tran.replace(regex={r'_prot$': '', r'_tran': ''})

# reverse mapping of gene name back onto uniprot id
prot_tran_annot = pd.merge(prot_map, prot_tran, how="right", left_on="val", right_on="proteome").sort_values(by=["proteome", "translatome"], ignore_index=True)
print(prot_tran_annot.head())
prot_tran_annot = pd.merge(tran_map, prot_tran_annot, how="right", left_on="val", right_on="translatome").sort_values(by=["proteome", "translatome"], ignore_index=True)
prot_tran_annot.columns = ["tran_uniprot", "tran_gene", "prot_uniprot", "prot_gene", "proteome", "translatome", "correlation"]
print(prot_tran_annot.head())
prot_tran_annot = prot_tran_annot.replace(regex={r'_prot$': '', r'_tran$': '', r'__FEATUREID$': ''})
prot_tran_annot.drop(["proteome", "translatome"], axis=1, inplace=True)
print(prot_tran_annot.head())

# use panther annotations
panther_annot = panther_annotations[[1,2,4]].replace(regex={r',.*$': ''})
panther_annot.columns = [1,2,3]

# use covid annotations
covid_annot = covid_annotations.copy()
covid_annot.replace(regex={r'_HUMAN$': ''}, inplace=True)
covid_annot.drop(["Status", "Gene names", "Entry name", "Length"], axis=1, inplace=True)
covid_annot.columns = [1, 2, 3]
covid_annot

# merge annotations
annot = pd.concat([panther_annot, covid_annot], ignore_index=True)

# annotate genes
prot_tran_annot = pd.merge(annot, prot_tran_annot, how="right", left_on=1, right_on="prot_uniprot")
prot_tran_annot.columns = ["tmp2", "prot_annotation", "prot_category", "tran_uniprot", "tran_gene", "prot_uniprot", "prot_gene", "correlation"]
prot_tran_annot = pd.merge(annot, prot_tran_annot, how="right", left_on=1, right_on="tran_uniprot")
prot_tran_annot.columns = ["tmp1", "tran_annotation", "tran_category", "tmp2", "prot_annotation", "prot_category", "tran_uniprot", "tran_gene", "prot_uniprot", "prot_gene", "correlation"]
prot_tran_annot.drop(["tmp1", "tmp2"], axis=1, inplace=True)
cols = ["tran_uniprot", "tran_annotation", "tran_category", "tran_gene", "prot_uniprot", "prot_annotation", "prot_category", "prot_gene", "correlation"]
prot_tran_annot = prot_tran_annot[cols]
prot_tran_annot.sort_values(by="correlation", ascending=False, ignore_index=True, inplace=True)
prot_tran_annot.to_csv("multiblock_correlations_annotated.tsv", sep="\t")

In [4]:
# export single omics proteomics with annotations
prot_splsda = pd.read_csv(prot_splsda_infile, sep="\t")
prot_splsda.reset_index(inplace=True)
prot_splsda.replace(regex={r'_prot$': ''}, inplace=True)
prot_splsda.replace(regex={r'_tran$': ''}, inplace=True)
prot_splsda.replace(regex={r'__FEATUREID$': ''}, inplace=True)
prot_splsda.set_index("index", inplace=True)
prot_splsda

prot_splsda_annot = pd.merge(annot, prot_splsda, how="right", left_on=1, right_on="index")
prot_splsda_annot.columns = ["index", "name", "annotation"] + prot_splsda_annot.columns.to_list()[3:]
prot_splsda_annot.sort_values(by="importance", ascending=False, ignore_index=True, inplace=True)
prot_splsda_annot.set_index("index", inplace=True)
prot_splsda_annot.to_csv("proteome_splsda.tsv", sep="\t")
prot_splsda_annot

,name,annotation,Control_10h,Control_24h,Control_2h,Control_6h,Virus_10h,Virus_24h,Virus_2h,Virus_6h,...,Contrib.Control_2h,Contrib.Control_6h,Contrib.Virus_10h,Contrib.Virus_24h,Contrib.Virus_2h,Contrib.Virus_6h,Contrib,GroupContrib,color,importance
index,,,,,,,,,,,,,,,,,,,,,
P08174,Complement decay-accelerating factor;CD55;orth...,NaN,0.519820,0.013958,0.558856,-2.479036,0.662507,-0.314339,0.768439,0.191540,...,False,False,False,False,True,False,False,Virus_2h,black,0.793975
Q16719,Kynureninase;KYNU;ortholog,hydrolase(PC00121),0.656747,0.311750,0.803123,-2.509965,0.474387,-0.327446,0.218274,0.165729,...,True,False,False,False,False,False,False,Control_2h,#C2C2C2,0.368682
Q99988,NaN,NaN,-0.773852,-1.221887,1.355051,0.397736,-1.156558,-0.711237,1.588026,0.260853,...,False,False,False,False,True,False,False,Virus_2h,black,0.359381
Q8WVV9_P14866,NaN,NaN,-1.208401,-1.085086,0.919806,0.180927,-1.364740,-0.066748,1.120907,0.627398,...,False,False,False,False,True,False,False,Virus_2h,black,0.343092
Q2UVX4,NaN,NaN,-0.192311,-0.355171,1.092389,-1.992105,0.225023,0.131557,1.182446,-0.172031,...,False,False,False,False,True,False,False,Virus_2h,black,0.284355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q71RC2,NaN,NaN,0.896134,0.500945,-1.020269,-0.273502,1.500922,-0.019695,-1.014450,-0.437232,...,False,False,True,False,False,False,False,Virus_10h,#CC79A7,-0.346225
P04183,NaN,NaN,-0.714823,2.420303,-0.278035,-0.482255,-0.578664,0.392170,-0.281566,-0.478009,...,False,False,False,False,False,False,False,Control_24h,#F68B33,-0.376758
P31350,NaN,NaN,-0.608445,2.650162,-0.049620,-0.474293,-0.641835,-0.156818,-0.205299,-0.394859,...,False,False,False,False,False,False,False,Control_24h,#F68B33,-0.389780


In [5]:
# export single omics proteomics with annotations
tran_splsda = pd.read_csv(tran_splsda_infile, sep="\t")
tran_splsda.reset_index(inplace=True)
tran_splsda.replace(regex={r'_prot$': ''}, inplace=True)
tran_splsda.replace(regex={r'_tran$': ''}, inplace=True)
tran_splsda.replace(regex={r'__FEATUREID$': ''}, inplace=True)
tran_splsda.set_index("index", inplace=True)
tran_splsda

tran_splsda_annot = pd.merge(annot, tran_splsda, how="right", left_on=1, right_on="index")
tran_splsda_annot.columns = ["index", "name", "annotation"] + tran_splsda_annot.columns.to_list()[3:]
tran_splsda_annot.sort_values(by="importance", ascending=False, ignore_index=True, inplace=True)
tran_splsda_annot.set_index("index", inplace=True)
tran_splsda_annot.to_csv("translatome_splsda.tsv", sep="\t")
tran_splsda_annot

,name,annotation,Control_10h,Control_24h,Control_2h,Control_6h,Virus_10h,Virus_24h,Virus_2h,Virus_6h,...,Contrib.Control_2h,Contrib.Control_6h,Contrib.Virus_10h,Contrib.Virus_24h,Contrib.Virus_2h,Contrib.Virus_6h,Contrib,GroupContrib,color,importance
index,,,,,,,,,,,,,,,,,,,,,
P11142,Heat shock cognate 71 kDa protein;HSPA8;ortholog,NaN,1.773624,-0.810417,-1.077815,0.705951,0.784379,-0.688406,-1.061882,0.615859,...,False,False,False,False,False,False,False,Control_10h,#388ECC,0.459400
P08865,40S ribosomal protein SA;RPSA;ortholog,ribosomal protein(PC00202),2.082822,-0.579825,-0.789736,0.320292,0.767054,-0.881572,-0.990181,0.279845,...,False,False,False,False,False,False,False,Control_10h,#388ECC,0.396760
Q14978,NaN,NaN,1.686611,-0.546829,-0.836080,0.829102,0.417661,-1.239525,-0.948608,0.778264,...,False,False,False,False,False,False,False,Control_10h,#388ECC,0.349158
P40616,NaN,NaN,-0.608382,0.458432,0.557968,1.289721,-1.698614,-0.073766,-0.606656,0.178448,...,False,True,False,False,False,False,False,Control_6h,#009E73,0.282703
P19338,Nucleolin;NCL;ortholog,NaN,1.769154,-0.522630,-0.681460,0.798107,0.295510,-1.024305,-1.142202,0.679641,...,False,False,False,False,False,False,False,Control_10h,#388ECC,0.280191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q7Z3J2,VPS35 endosomal protein sorting factor-like;VP...,NaN,-0.051014,-0.771387,-0.097190,-0.381565,-0.310444,2.300429,-0.498069,-0.641227,...,False,False,False,True,False,False,False,Virus_24h,#F0E442,-0.309439
Q9Y281,NaN,NaN,-0.236979,-0.513830,-0.003641,-0.317454,-0.316224,2.440445,-0.605496,-0.437982,...,False,False,False,True,False,False,False,Virus_24h,#F0E442,-0.309923
Q07021,Complement component 1 Q subcomponent-binding ...,NaN,0.212073,-0.388812,-0.613739,-0.443112,-0.400930,2.592492,-0.485549,-0.524936,...,False,False,False,True,False,False,False,Virus_24h,#F0E442,-0.344041
